Importing all necessary packages

In [0]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

Downloading the training dataset. Also loading the data in tensors.

In [4]:
transform=transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

trainset=torchvision.datasets.CIFAR10(root='./data', train=True,
                                    download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes=('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

Failed download. Trying https -> http instead. Downloading http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ./data/cifar-10-python.tar.gz


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Here the neural net is defined.

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, (5,5))
        self.pool = nn.MaxPool2d((2,2), 2)
        self.conv2 = nn.Conv2d(6, 16, (5,5))
        self.fc1 = nn.Linear(16 * (5*5), 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * (5*5))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

Setting the loss as Cross-Entropy Loss and optimizer as momentum. I am using stochastic gradient descent here.

In [37]:
# Classification Cross-Entropy loss and SGD with momentum
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(),lr=10**(-3),momentum=0.9)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

Training my network here.

In [38]:
for epoch in range(12): # no of epochs

    running_loss=0.0
    for i,data in enumerate(trainloader,0):
          inputs, labels = data[0].to(device), data[1].to(device)
          # print(inputs.shape)
          # print(labels.shape)
          # forward + backward + optimize
          optimizer.zero_grad()
          outputs=net(inputs)
          # print(outputs.shape)
          loss=criterion(outputs,labels)
          loss.backward()
          optimizer.step()
          running_loss+=loss.item()
          if i%2000==1999:
                if i%12000==11999:
                  print('Epoch:',epoch+1,', loss:',running_loss/2000)
                running_loss=0
print('Model trained :)')

Epoch: 1 , loss: 1.463351932182908
Epoch: 2 , loss: 1.2970442164391278
Epoch: 3 , loss: 1.181214131101966
Epoch: 4 , loss: 1.0958619959056377
Epoch: 5 , loss: 1.0333775982409716
Epoch: 6 , loss: 0.9750001984983683
Epoch: 7 , loss: 0.9444688875377178
Epoch: 8 , loss: 0.9148950591981411
Epoch: 9 , loss: 0.8738548803329468
Epoch: 10 , loss: 0.8416195877045393
Epoch: 11 , loss: 0.84942894038558
Epoch: 12 , loss: 0.7971686856001615
Model trained :)


Testing my network with all the test data.

In [39]:
correct=0
total=0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs=net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print("10000 images in total")
print("Accuracy of network=",correct*100/total)

10000 images in total
Accuracy of network= 62.76
